In [ ]:
import numpy as np
import math

In [18]:
#Вариант 4

def LoadTest(num):
  if num == 1: #хорошо обусловленная
    A = np.array([
        [-62.8000, -6.9700, 7.7300, 0.0000],
        [0.0000, 49.4000, -3.5600, -4.5700],
        [-7.1100, -5.9700, 97.6000, -7.5800],
        [6.8100, -1.2900, -7.9500, 42.2000]
    ], dtype = "float64")
    b = np.array([-221.3100, -523.8800, 1121.6300, 294.6700], dtype = "float64")
    sol = np.array([6, -9, 12, 8], dtype = "float64")
  if num == 2: #плохо обусловленная
    A = np.array([
        [0.0500, -10.4550, -5.3550, 1.7850],
        [0.0000, 12.6410, 6.3210, -2.1070],
        [0.3600, 15.4650, 7.0850, -2.3750],
        [1.0800, 120.4410, 58.2810, -19.4670]
    ], dtype = "float64")
    b = np.array([26.2450+0.01, -29.4990-0.01, -24.5950+0.01, -246.5790-0.01], dtype = "float64")
    sol = np.array([20, 1, -6, 2], dtype = "float64")
  return A, b, sol


In [ ]:
def house(x):
  n=len(x); mu=np.linalg.norm(x); v=np.copy(x)
  if mu:
    beta=x[0]+(1.0 if x[0] >= 0 else -1.0)*mu
    v[1:]/=beta
  v[0]=1.0
  return v

def row_house(A,v):
  beta = -2.0 / np.dot(v,v)
  w = beta * np.matmul(A.T,v)
  A+=np.outer(v,w)

def col_house(A,v):
  beta = -2.0 / np.dot(v,v)
  w = beta * np.matmul(A,v)
  A+=np.outer(w,v)

def house_QR(A):
  m,n=A.shape
  for j in range(n):
    v=house(A[j:,j])
    row_house(A[j:,j:],v)
    if j<m-1:
      A[j+1:,j]=v[1:]

def rev_subs(U,b):
  n=len(b)
  b[n-1]/=U[n-1,n-1]
  for i in range(n-2,-1,-1):
    b[i]=(b[i]-np.dot(U[i,i+1:],b[i+1:]))/U[i,i]

def row_house_vec(x,v):
  beta = -2.0 * np.dot(v,x) / np.dot(v,v)
  x += beta * v

def solve_house(A,b):
  house_QR(A)
  _,n=A.shape
  v=np.zeros(n)
  for j in range(n):
    v[j]=1.0; v[j+1:]=A[j+1:,j]
    row_house_vec(b[j:],v[j:])
  rev_subs(A,b)

In [19]:
A, b, sol = LoadTest(1)

solve_house(A, b)
print("Решение: ", b)
print("Абсолютная ошибка: ", b - sol)
print("Относительная ошибка: ", (b - sol) / np.linalg.norm(b-sol))
A, f, sol = LoadTest(1)

print("\nНевязка: ", A@b - f)
print("1-норма: ", np.linalg.norm(A@b - f, ord = 1))
print("inf-норма: ", np.linalg.norm(A@b - f, ord = math.inf))
print("\nЧисло обусловленности системы: ")
print("1-норма: ", np.linalg.cond(A, p = 1))
print("inf-норма: ", np.linalg.cond(A, p = math.inf))

A, b, sol = LoadTest(2)

solve_house(A, b)
print("\nРешение: ", b)
print("Абсолютная ошибка: ", b - sol)
print("Относительная ошибка: ", (b - sol) / np.linalg.norm(b-sol))

A, f, sol = LoadTest(2)

print("\nНевязка: ", A@b - f)
print("1-норма: ", np.linalg.norm(A@b - f, ord = 1))
print("inf-норма: ", np.linalg.norm(A@b - f, ord = math.inf))
print("\nЧисло обусловленности системы: ")
print("1-норма: ", np.linalg.cond(A, p = 1))
print("inf-норма: ", np.linalg.cond(A, p = math.inf))

Решение:  [ 6. -9. 12.  8.]
Абсолютная ошибка:  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.77635684e-15]
Относительная ошибка:  [ 0.  0.  0. -1.]

Невязка:  [ 2.84217094e-14  0.00000000e+00 -4.54747351e-13 -5.68434189e-14]
1-норма:  5.400124791776761e-13
inf-норма:  4.547473508864641e-13

Число обусловленности системы: 
1-норма:  3.3379041290652394
inf-норма:  3.49769768291856

Решение:  [ 683.19999749 -129.46666617 1995.6666591  5224.26664694]
Абсолютная ошибка:  [ 663.19999749 -130.46666617 2001.6666591  5222.26664694]
Относительная ошибка:  [ 0.11772568 -0.02315934  0.35531901  0.92701279]

Невязка:  [-2.62190269e-12  7.26174676e-12  8.70414851e-13  7.33280103e-12]
1-норма:  1.808686533877335e-11
inf-норма:  7.332801033044234e-12

Число обусловленности системы: 
1-норма:  401918895.2910938
inf-норма:  552318536.5954505


In [ ]:
A, _, _ = LoadTest(2)
x = []
def house_inverse(A):
  _,n = A.shape
  temp = A
  e = np.eye(n)
  temp_e = e
  for i in range(n):
    A = np.copy(temp)
    e = np.copy(temp_e)
    solve_house(A, e[i])
    x.append(e[i])
  return np.transpose(np.array(x))

A_inv = house_inverse(A)
A, _, _ = LoadTest(2)

display(A_inv.dot(A))

array([[ 9.99999999e-01, -4.05784531e-08, -1.88219494e-08,
         5.93150571e-09],
       [-3.73683898e-12,  1.00000000e+00,  1.86456645e-11,
         5.89915696e-11],
       [-5.14828479e-12, -6.38106311e-09,  9.99999996e-01,
         4.54505951e-11],
       [-7.96146564e-12, -1.43737735e-08, -8.72108779e-09,
         1.00000000e+00]])